# Train using attention model

Reference: https://www.kaggle.com/truocpham/oob-cuda-gru-attention


In [26]:
import pandas as pd
from itertools import chain

import sklearn
import keras

#### BEGIN Attention Model ####
from keras.models import Sequential,Model
from keras.layers import *
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
#### END Attention Model ####


import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

# Read Dataset

In [2]:
filename_train = './datasets/train.json'
filename_test = './datasets/test.json'

train = pd.read_json(filename_train)
test = pd.read_json(filename_test)

In [3]:
train.shape

(1195, 5)

In [4]:
train.head()

,audio_embedding,end_time_seconds_youtube_clip,is_turkey,start_time_seconds_youtube_clip,vid_id
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",70,0,60,kDCk3hLIVXo
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",40,1,30,DPcGzqHoo7Y
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",240,1,230,7yM63MTHh5k
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",520,1,510,luG3RmUAxxM
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",10,0,0,PIm3cjxTpOk


# Retrive Audio Embedding and Flatten it into 1D

In [5]:
embeddings = pd.DataFrame(list(train['audio_embedding'].map(lambda x: chain.from_iterable(x))))

In [6]:
embeddings.shape

(1195, 1280)

In [7]:
embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,172,34,216,110,208,46,95,66,161,125,...,0.0,135.0,133.0,151.0,0.0,3.0,206.0,101.0,104.0,255.0
1,169,20,165,102,205,62,110,103,211,187,...,0.0,0.0,119.0,205.0,27.0,151.0,226.0,44.0,0.0,255.0
2,148,8,138,60,237,48,121,108,145,177,...,0.0,62.0,79.0,204.0,0.0,74.0,243.0,255.0,95.0,255.0
3,151,0,162,88,171,71,47,90,179,190,...,0.0,255.0,207.0,52.0,178.0,129.0,186.0,0.0,0.0,255.0
4,162,17,187,111,211,105,92,67,203,152,...,62.0,224.0,15.0,172.0,0.0,2.0,255.0,144.0,34.0,255.0


# Check whether NULL Cell exist

In [8]:
embeddings.isnull().any().any()

True

In [9]:
embeddings = embeddings.fillna(-1)

# Define Model

# Prepare Train Dataset

In [10]:
xtrain = [k for k in train['audio_embedding']]
test_data = test['audio_embedding'].tolist()

In [11]:
print(len(xtrain))
print(len(test_data))

1195
1196


In [12]:
ytrain = train['is_turkey'].values

# Check the Min/Max Dimension

In [13]:
max([len(features) for features in xtrain])

10

In [14]:
min([len(features) for features in xtrain])

2

# Pad audio features

In [15]:
x_train = keras.preprocessing.sequence.pad_sequences(xtrain, maxlen=10)

In [16]:
x_train.shape

(1195, 10, 128)

In [17]:
x_train.max()

255

In [18]:
x_train.min()

0

In [19]:
type(x_train)

numpy.ndarray

In [20]:
np.isnan(x_train).any()

False

# Convert ytrain to np.array

In [21]:
y_train = np.array(ytrain)

In [22]:
y_train.shape

(1195,)

# Delcare Model

In [23]:
# https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

# Model w/o Bidirectional Layer

In [24]:
def get_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(10, 128)))
#     model.add(Bidirectional(RNN(64, activation='relu', return_sequences=True)))
#     model.add(Bidirectional(GRU(128, dropout=0.4, recurrent_dropout=0.4, activation='relu', return_sequences=True)))
    model.add(GRU(128, dropout=0.4, recurrent_dropout=0.4, activation='relu', return_sequences=True))
    model.add(Attention(10))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


input_dim = x_train.shape
# model = get_model(input_dim)
model = get_model()


model.summary()

W0717 12:44:58.951398 140206139774656 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 12:44:58.969661 140206139774656 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 12:44:59.040005 140206139774656 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0717 12:44:59.054923 140206139774656 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Pleas

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 10, 128)           512       
_________________________________________________________________
gru_1 (GRU)                  (None, 10, 128)           98688     
_________________________________________________________________
attention_1 (Attention)      (None, 128)               138       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 99,467
Trainable params: 99,211
Non-trainable params: 256
_________________________________________________________________


In [27]:

scores = []

# sklearn.model_selection.KFold
kf = StratifiedKFold(n_splits=10,
          shuffle=True,
          random_state=42069)

for index_fold, (index_train, index_val) in enumerate(kf.split(x_train, y_train)):
    x_train_f = x_train[index_train]
    y_train_f = y_train[index_train]
    
    x_val_f = x_train[index_val]
    y_val_f = y_train[index_val]
    
    # train
    model.fit(x_train_f, y_train_f, batch_size=256, epochs=16, verbose=1, validation_data=(x_val_f, y_val_f))
    
    print()
    
    # compute ROC-AUC score
    pred_val = model.predict([x_val_f], batch_size=512)
    
    score_auc = sklearn.metrics.roc_auc_score(y_val_f, pred_val)
    scores.append(score_auc)
    
    print('Fold {}: score={}'.format(index_fold, score_auc))
    
    
print('Average AUC score: {}'.format(np.mean(scores)))

Train on 1074 samples, validate on 121 samples
Epoch 1/16
1074/1074 [==============================] - 0s 209us/step - loss: 0.1219 - acc: 0.9590 - val_loss: 0.1138 - val_acc: 0.9669
Epoch 2/16
1074/1074 [==============================] - 0s 242us/step - loss: 0.1178 - acc: 0.9553 - val_loss: 0.1121 - val_acc: 0.9669
Epoch 3/16
1074/1074 [==============================] - 0s 202us/step - loss: 0.1089 - acc: 0.9590 - val_loss: 0.1116 - val_acc: 0.9669
Epoch 4/16
1074/1074 [==============================] - 0s 196us/step - loss: 0.1154 - acc: 0.9581 - val_loss: 0.1116 - val_acc: 0.9669
Epoch 5/16
1074/1074 [==============================] - 0s 233us/step - loss: 0.1011 - acc: 0.9628 - val_loss: 0.1117 - val_acc: 0.9669
Epoch 6/16
1074/1074 [==============================] - 0s 196us/step - loss: 0.1038 - acc: 0.9590 - val_loss: 0.1113 - val_acc: 0.9669
Epoch 7/16
1074/1074 [==============================] - 0s 195us/step - loss: 0.1019 - acc: 0.9628 - val_loss: 0.1116 - val_acc: 0.9669
E

1075/1075 [==============================] - 0s 188us/step - loss: 0.0421 - acc: 0.9814 - val_loss: 0.0271 - val_acc: 0.9833
Epoch 12/16
1075/1075 [==============================] - 0s 192us/step - loss: 0.0388 - acc: 0.9879 - val_loss: 0.0234 - val_acc: 0.9833
Epoch 13/16
1075/1075 [==============================] - 0s 196us/step - loss: 0.0347 - acc: 0.9870 - val_loss: 0.0235 - val_acc: 0.9833
Epoch 14/16
1075/1075 [==============================] - 0s 194us/step - loss: 0.0385 - acc: 0.9851 - val_loss: 0.0256 - val_acc: 0.9833
Epoch 15/16
1075/1075 [==============================] - 0s 198us/step - loss: 0.0303 - acc: 0.9888 - val_loss: 0.0274 - val_acc: 0.9833
Epoch 16/16
1075/1075 [==============================] - 0s 197us/step - loss: 0.0359 - acc: 0.9860 - val_loss: 0.0287 - val_acc: 0.9833

Fold 3: score=0.9997125610807703
Train on 1076 samples, validate on 119 samples
Epoch 1/16
1076/1076 [==============================] - 0s 196us/step - loss: 0.0316 - acc: 0.9861 - val_loss

1076/1076 [==============================] - 0s 199us/step - loss: 0.0213 - acc: 0.9916 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 6/16
1076/1076 [==============================] - 0s 191us/step - loss: 0.0174 - acc: 0.9935 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 7/16
1076/1076 [==============================] - 0s 190us/step - loss: 0.0171 - acc: 0.9954 - val_loss: 0.0014 - val_acc: 1.0000
Epoch 8/16
1076/1076 [==============================] - 0s 190us/step - loss: 0.0104 - acc: 0.9972 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 9/16
1076/1076 [==============================] - 0s 190us/step - loss: 0.0121 - acc: 0.9944 - val_loss: 0.0014 - val_acc: 1.0000
Epoch 10/16
1076/1076 [==============================] - 0s 191us/step - loss: 0.0136 - acc: 0.9954 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 11/16
1076/1076 [==============================] - 0s 189us/step - loss: 0.0117 - acc: 0.9972 - val_loss: 0.0017 - val_acc: 1.0000
Epoch 12/16
1076/1076 [==============================] - 

# Model w/ Bidirectional Layer

In [51]:
# def get_model(input_dim, hidden_size=64, fc1_size=10, output_size=1, lr=1e-3):
    
#     # output shape = (None, seq_length, feature_size)
#     inputs = keras.layers.Input(input_dim[1:])

#     # output shape = (None, seq_length, hidden_size)
#     x_rnn = keras.layers.GRU(units=hidden_size)(inputs)
    
#     # output shape = (None, fc1_size)
#     x_attention = Attention(fc1_size)(x_rnn)
    
#     # output shape = (None, 1)
#     outputs = keras.layers.Dense(output_size, activation='sigmoid')(x_attention)
    
#     model = keras.models.Model(inputs=inputs, outputs=outputs)
    
#     optimizer = keras.optimizers.Adam(lr=lr)
    
#     model.compile(loss='binary_crossentropy',
#                   optimizer=optimizer,
#                  metrics=['accuracy'])
#     return model

def get_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(10, 128)))
#     model.add(Bidirectional(RNN(64, activation='relu', return_sequences=True)))
    model.add(Bidirectional(GRU(128, dropout=0.4, recurrent_dropout=0.4, activation='relu', return_sequences=True)))
    model.add(Attention(10))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


input_dim = x_train.shape
# model = get_model(input_dim)
model = get_model()


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 10, 128)           512       
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 256)           197376    
_________________________________________________________________
attention_4 (Attention)      (None, 256)               266       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 198,411
Trainable params: 198,155
Non-trainable params: 256
_________________________________________________________________


# 10-Fold Training

In [52]:

scores = []

# sklearn.model_selection.KFold
kf = StratifiedKFold(n_splits=10,
          shuffle=True,
          random_state=42069)

for index_fold, (index_train, index_val) in enumerate(kf.split(x_train, y_train)):
    x_train_f = x_train[index_train]
    y_train_f = y_train[index_train]
    
    x_val_f = x_train[index_val]
    y_val_f = y_train[index_val]
    
    # train
    model.fit(x_train_f, y_train_f, batch_size=256, epochs=16, verbose=1, validation_data=(x_val_f, y_val_f))
    
    print()
    
    # compute ROC-AUC score
    pred_val = model.predict([x_val_f], batch_size=512)
    
    score_auc = sklearn.metrics.roc_auc_score(y_val_f, pred_val)
    scores.append(score_auc)
    
    print('Fold {}: score={}'.format(index_fold, score_auc))
    
    
print('Average AUC score: {}'.format(np.mean(scores)))

Train on 1074 samples, validate on 121 samples
Epoch 1/16
1074/1074 [==============================] - 2s 2ms/step - loss: 0.6147 - acc: 0.6620 - val_loss: 0.3460 - val_acc: 0.9174
Epoch 2/16
1074/1074 [==============================] - 0s 389us/step - loss: 0.3145 - acc: 0.9134 - val_loss: 0.2029 - val_acc: 0.9421
Epoch 3/16
1074/1074 [==============================] - 0s 390us/step - loss: 0.2053 - acc: 0.9348 - val_loss: 0.1550 - val_acc: 0.9504
Epoch 4/16
1074/1074 [==============================] - 0s 399us/step - loss: 0.1614 - acc: 0.9348 - val_loss: 0.1430 - val_acc: 0.9587
Epoch 5/16
1074/1074 [==============================] - 0s 392us/step - loss: 0.1442 - acc: 0.9432 - val_loss: 0.1412 - val_acc: 0.9587
Epoch 6/16
1074/1074 [==============================] - 0s 403us/step - loss: 0.1415 - acc: 0.9451 - val_loss: 0.1386 - val_acc: 0.9587
Epoch 7/16
1074/1074 [==============================] - 0s 380us/step - loss: 0.1215 - acc: 0.9544 - val_loss: 0.1363 - val_acc: 0.9587
Epo

1075/1075 [==============================] - 0s 392us/step - loss: 0.0266 - acc: 0.9926 - val_loss: 0.0280 - val_acc: 0.9917
Epoch 12/16
1075/1075 [==============================] - 0s 397us/step - loss: 0.0259 - acc: 0.9907 - val_loss: 0.0318 - val_acc: 0.9917
Epoch 13/16
1075/1075 [==============================] - 0s 402us/step - loss: 0.0257 - acc: 0.9916 - val_loss: 0.0320 - val_acc: 0.9917
Epoch 14/16
1075/1075 [==============================] - 0s 387us/step - loss: 0.0204 - acc: 0.9926 - val_loss: 0.0294 - val_acc: 0.9917
Epoch 15/16
1075/1075 [==============================] - 0s 398us/step - loss: 0.0176 - acc: 0.9953 - val_loss: 0.0299 - val_acc: 0.9917
Epoch 16/16
1075/1075 [==============================] - 0s 397us/step - loss: 0.0223 - acc: 0.9935 - val_loss: 0.0320 - val_acc: 0.9917

Fold 3: score=0.9994251221615407
Train on 1076 samples, validate on 119 samples
Epoch 1/16
1076/1076 [==============================] - 0s 390us/step - loss: 0.0180 - acc: 0.9963 - val_loss

Epoch 5/16
1076/1076 [==============================] - 0s 396us/step - loss: 0.0091 - acc: 0.9972 - val_loss: 4.8713e-04 - val_acc: 1.0000
Epoch 6/16
1076/1076 [==============================] - 0s 390us/step - loss: 0.0086 - acc: 0.9981 - val_loss: 6.1685e-04 - val_acc: 1.0000
Epoch 7/16
1076/1076 [==============================] - 0s 387us/step - loss: 0.0066 - acc: 0.9981 - val_loss: 6.2527e-04 - val_acc: 1.0000
Epoch 8/16
1076/1076 [==============================] - 0s 385us/step - loss: 0.0055 - acc: 0.9981 - val_loss: 6.0206e-04 - val_acc: 1.0000
Epoch 9/16
1076/1076 [==============================] - 0s 405us/step - loss: 0.0054 - acc: 1.0000 - val_loss: 5.5121e-04 - val_acc: 1.0000
Epoch 10/16
1076/1076 [==============================] - 0s 400us/step - loss: 0.0077 - acc: 0.9981 - val_loss: 4.5831e-04 - val_acc: 1.0000
Epoch 11/16
1076/1076 [==============================] - 0s 390us/step - loss: 0.0081 - acc: 0.9972 - val_loss: 4.7899e-04 - val_acc: 1.0000
Epoch 12/16
1076/1

# Model w/o Attention, but w/ GlobalAveragePooling

In [29]:
def get_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(10, 128)))
#     model.add(Bidirectional(RNN(64, activation='relu', return_sequences=True)))
#     model.add(Bidirectional(GRU(128, dropout=0.4, recurrent_dropout=0.4, activation='relu', return_sequences=True)))
    model.add(GRU(128, dropout=0.4, recurrent_dropout=0.4, activation='relu', return_sequences=True))
#     model.add(Attention(10))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


input_dim = x_train.shape
# model = get_model(input_dim)
model = get_model()


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 10, 128)           512       
_________________________________________________________________
gru_3 (GRU)                  (None, 10, 128)           98688     
_________________________________________________________________
global_average_pooling1d_2 ( (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 99,329
Trainable params: 99,073
Non-trainable params: 256
_________________________________________________________________


In [30]:

scores = []

# sklearn.model_selection.KFold
kf = StratifiedKFold(n_splits=10,
          shuffle=True,
          random_state=42069)

for index_fold, (index_train, index_val) in enumerate(kf.split(x_train, y_train)):
    x_train_f = x_train[index_train]
    y_train_f = y_train[index_train]
    
    x_val_f = x_train[index_val]
    y_val_f = y_train[index_val]
    
    # train
    model.fit(x_train_f, y_train_f, batch_size=256, epochs=16, verbose=1, validation_data=(x_val_f, y_val_f))
    
    print()
    
    # compute ROC-AUC score
    pred_val = model.predict([x_val_f], batch_size=512)
    
    score_auc = sklearn.metrics.roc_auc_score(y_val_f, pred_val)
    scores.append(score_auc)
    
    print('Fold {}: score={}'.format(index_fold, score_auc))
    
    
print('Average AUC score: {}'.format(np.mean(scores)))

Train on 1074 samples, validate on 121 samples
Epoch 1/16
1074/1074 [==============================] - 1s 1ms/step - loss: 0.6525 - acc: 0.5931 - val_loss: 0.4091 - val_acc: 0.9256
Epoch 2/16
1074/1074 [==============================] - 0s 195us/step - loss: 0.3791 - acc: 0.8985 - val_loss: 0.2443 - val_acc: 0.9587
Epoch 3/16
1074/1074 [==============================] - 0s 199us/step - loss: 0.2548 - acc: 0.9181 - val_loss: 0.1729 - val_acc: 0.9587
Epoch 4/16
1074/1074 [==============================] - 0s 189us/step - loss: 0.2055 - acc: 0.9171 - val_loss: 0.1432 - val_acc: 0.9587
Epoch 5/16
1074/1074 [==============================] - 0s 190us/step - loss: 0.1639 - acc: 0.9376 - val_loss: 0.1314 - val_acc: 0.9669
Epoch 6/16
1074/1074 [==============================] - 0s 194us/step - loss: 0.1682 - acc: 0.9376 - val_loss: 0.1281 - val_acc: 0.9669
Epoch 7/16
1074/1074 [==============================] - 0s 188us/step - loss: 0.1503 - acc: 0.9432 - val_loss: 0.1256 - val_acc: 0.9587
Epo

1075/1075 [==============================] - 0s 199us/step - loss: 0.0628 - acc: 0.9758 - val_loss: 0.0494 - val_acc: 0.9833
Epoch 12/16
1075/1075 [==============================] - 0s 216us/step - loss: 0.0476 - acc: 0.9814 - val_loss: 0.0505 - val_acc: 0.9833
Epoch 13/16
1075/1075 [==============================] - 0s 196us/step - loss: 0.0472 - acc: 0.9870 - val_loss: 0.0514 - val_acc: 0.9833
Epoch 14/16
1075/1075 [==============================] - 0s 203us/step - loss: 0.0480 - acc: 0.9767 - val_loss: 0.0493 - val_acc: 0.9833
Epoch 15/16
1075/1075 [==============================] - 0s 198us/step - loss: 0.0447 - acc: 0.9805 - val_loss: 0.0496 - val_acc: 0.9833
Epoch 16/16
1075/1075 [==============================] - 0s 196us/step - loss: 0.0420 - acc: 0.9870 - val_loss: 0.0520 - val_acc: 0.9833

Fold 3: score=0.999137683242311
Train on 1076 samples, validate on 119 samples
Epoch 1/16
1076/1076 [==============================] - 0s 193us/step - loss: 0.0494 - acc: 0.9823 - val_loss:

1076/1076 [==============================] - 0s 200us/step - loss: 0.0241 - acc: 0.9916 - val_loss: 0.0019 - val_acc: 1.0000
Epoch 6/16
1076/1076 [==============================] - 0s 197us/step - loss: 0.0172 - acc: 0.9954 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 7/16
1076/1076 [==============================] - 0s 201us/step - loss: 0.0214 - acc: 0.9954 - val_loss: 0.0024 - val_acc: 1.0000
Epoch 8/16
1076/1076 [==============================] - 0s 198us/step - loss: 0.0189 - acc: 0.9926 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 9/16
1076/1076 [==============================] - 0s 193us/step - loss: 0.0190 - acc: 0.9963 - val_loss: 0.0017 - val_acc: 1.0000
Epoch 10/16
1076/1076 [==============================] - 0s 195us/step - loss: 0.0171 - acc: 0.9963 - val_loss: 0.0023 - val_acc: 1.0000
Epoch 11/16
1076/1076 [==============================] - 0s 193us/step - loss: 0.0200 - acc: 0.9916 - val_loss: 0.0025 - val_acc: 1.0000
Epoch 12/16
1076/1076 [==============================] - 